In [8]:
import aiohttp
import json

In [16]:
tools = [
{
    "name": "cleanup",
    "description": "Cleanup all managed resources",
    "inputSchema": {
        "type": "object",
        "properties": {}
    }
},
{
    "name": "kubectl_get",
    "description": "Get or list Kubernetes resources by resource type, name, and optionally namespace",
    "inputSchema": {
        "type": "object",
        "properties": {
            "resourceType": {
                "type": "string",
                "description": "Type of resource to get (e.g., pods, deployments, services, configmaps, events, etc.)"
            },
            "name": {
                "type": "string",
                "description": "Name of the resource (optional - if not provided, lists all resources of the specified type)"
            },
            "namespace": {
                "type": "string",
                "description": "Namespace of the resource (optional - defaults to 'default' for namespaced resources)",
                "default": "default"
            },
            "output": {
                "type": "string",
                "enum": ["json", "yaml", "wide", "name", "custom"],
                "description": "Output format",
                "default": "json"
            },
            "allNamespaces": {
                "type": "boolean",
                "description": "If true, list resources across all namespaces",
                "default": False
            },
            "labelSelector": {
                "type": "string",
                "description": "Filter resources by label selector (e.g. 'app=nginx')"
            },
            "fieldSelector": {
                "type": "string",
                "description": "Filter resources by field selector (e.g. 'metadata.name=my-pod')"
            },
            "sortBy": {
                "type": "string",
                "description": "Sort events by a field (default: lastTimestamp). Only applicable for events."
            }
        },
        "required": ["resourceType", "name", "namespace"]
    }
},
{
    "name": "kubectl_apply",
    "description": "Apply a Kubernetes YAML manifest from a string or file",
    "inputSchema": {
            "type": "object",
            "properties": {
                "manifest": {
                    "type": "string",
                    "description": "YAML manifest to apply"
                },
                "filename": {
                    "type": "string",
                    "description": "Path to a YAML file to apply (optional - use either manifest or filename)"
                },
                "namespace": {
                    "type": "string",
                    "description": "Namespace to apply the resource to (optional)",
                    "default": "default"
                },
                "dryRun": {
                    "type": "boolean",
                    "description": "If true, only validate the resource, don't apply it",
                    "default": False
                },
                "force": {
                    "type": "boolean",
                    "description": "If true, immediately remove resources from API and bypass graceful deletion",
                    "default": False
                }
            },
            "required": []
    }
},
{
    "name": "kubectl_describe",
    "description": "Describe Kubernetes resources by resource type, name, and optionally namespace",
    "inputSchema": {
        "type": "object",
        "properties": {
            "resourceType": {
                "type": "string",
                "description": "Type of resource to describe (e.g., pods, deployments, services, etc.)"
            },
            "name": {
                "type": "string",
                "description": "Name of the resource to describe"
            },
            "namespace": {
                "type": "string",
                "description": "Namespace of the resource (optional - defaults to 'default' for namespaced resources)",
                "default": "default"
            },
            "allNamespaces": {
                "type": "boolean",
                "description": "If true, describe resources across all namespaces",
                "default": False
            }
        },
        "required": ["resourceType", "name"]
    }
}

]

In [ ]:
async def call_llm(messages) -> str:
        """Call your self-hosted LLM"""
        try:
            async with aiohttp.ClientSession() as session:
                payload = {
                    "model": "Meta-Llama-3.3-70B-Instruct",
                    "messages": messages,
                    "temperature": 0.1,
                    "max_tokens": 2000
                }
                
                async with session.post(
                    f"{ os.getenv("LLM_BASE_URL") }/v1/chat/completions",
                    json=payload,
                    headers={"Content-Type": "application/json"}
                ) as response:
                    if response.status == 200:
                        result = await response.json()
                        return result["choices"][0]["message"]["content"]
                    else:
                        error_text = await response.text()
                        raise Exception(f"LLM API error: {response.status} - {error_text}")
                        
        except Exception as e:
            print(f"❌ LLM call failed: {e}")
            return "Sorry, I couldn't process your request due to an LLM error."

In [22]:
def create_system_prompt() -> str:
        """Create system prompt with available Kubernetes tools"""
        tools_description = ""
        for tool in tools:
        #     print(f'{tool["name"]}: {tool["description"]}')
            tools_description += f"- {tool['name']}: {tool['description']}\n"
            if "inputSchema" in tool and tool["inputSchema"]:
                tools_description += f"  Parameters: {json.dumps(tool["inputSchema"], indent=2)}\n"
        
        return f"""You are a Kubernetes expert assistant. You can help users manage their Kubernetes cluster.

Available Kubernetes operations:
{tools_description}

When users ask about Kubernetes operations:
1. Understand their intent
2. Choose the appropriate tool from the available options
3. Call the tool with proper parameters
4. Provide clear explanations of the results

If you need to perform a Kubernetes operation, use the available tools. Format your tool calls as JSON.

Example tool call format:
{{"tool_name": "kubectl_get", "parameters": {{"resourceType": "pods", "namespace": "default"}}}}
"""

In [23]:
messages = [
            {"role": "system", "content": create_system_prompt()},
            {"role": "user", "content": "list all pods in sre-test namespace"}
        ]

In [26]:
await call_llm(messages=messages)

'To list all pods in the `sre-test` namespace, I will use the `kubectl_get` tool. Here is the JSON call:\n\n```json\n{\n  "tool_name": "kubectl_get",\n  "parameters": {\n    "resourceType": "pods",\n    "namespace": "sre-test",\n    "allNamespaces": false\n  }\n}\n```\n\nThis will retrieve a list of all pods in the `sre-test` namespace. Let me execute this command.'